In [1]:
!pip install thop

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.optim import SGD
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
import os
import sys
import pickle
from datetime import datetime
from google.colab import drive
import random

def set_random_seeds(seed=42):
    """Set random seeds for reproducibility across all libraries."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seeds set to {seed} for reproducibility")


set_random_seeds(84)


drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/3_DL_Project1_CIFAR10'
os.makedirs(DRIVE_PATH, exist_ok=True)
print(f"Google Drive mounted. Files will be saved to {DRIVE_PATH}")

# MODEL ARCHITECTURE COMPONENTS
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels//reduction, 1, bias=True),
            nn.SiLU(inplace=True),
            nn.Conv2d(channels//reduction, channels, 1, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x): return x * self.se(x)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, se=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se = SEBlock(out_channels) if se else None
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.silu(self.bn1(self.conv1(x)))
        if self.se: out = self.se(out)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.silu(out)

class ResNet(nn.Module):
    def __init__(self, num_blocks, num_channels=64, num_classes=10):
        super().__init__()
        self.in_channels = num_channels
        self.conv1 = nn.Conv2d(3, num_channels, 3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.layer1 = self._make_layer(num_channels, num_blocks[0], 1)
        self.layer2 = self._make_layer(num_channels*2, num_blocks[1], 2)
        self.layer3 = self._make_layer(num_channels*4, num_blocks[2], 2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(num_channels*4 * BasicBlock.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_channels, out_channels, stride, se=True))
            self.in_channels = out_channels * BasicBlock.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.silu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        return self.linear(out.view(out.size(0), -1))

# EMA MODEL IMPLEMENTATION
class ModelEMA:
    """ Model Exponential Moving Average """
    def __init__(self, model, decay=0.9999, device=None):
        self.ema = {k: v.clone().detach() for k, v in model.state_dict().items()}
        self.decay = decay
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.ema = {k: v.to(device) for k, v in self.ema.items()}
        self.model = model
        self.training_mode = False

    def update(self, model):
        with torch.no_grad():
            for k, v in model.state_dict().items():
                if v.dtype.is_floating_point:
                    self.ema[k] = self.ema[k] * self.decay + v.detach() * (1 - self.decay)

    def apply(self):
        self.training_mode = self.model.training
        self.ema_state_dict = self.model.state_dict()
        self.model.load_state_dict({k: v.clone() for k, v in self.ema.items()})
        self.model.eval()

    def restore(self):
        if self.training_mode:
            self.model.load_state_dict(self.ema_state_dict)
            self.model.train()

# MIXUP/CUTMIX IMPLEMENTATION
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def cutmix_data(x, y, alpha=1.0):
    '''Returns cutmix inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    # lambda exactly matches pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))

    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# CUTOUT AUGMENTATION
class Cutout:
    def __init__(self, n_holes=1, length=16):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.shape[1], img.shape[2]
        mask = np.ones((h, w), np.float32)
        for _ in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
            y1 = np.clip(y - self.length//2, 0, h)
            y2 = np.clip(y + self.length//2, 0, h)
            x1 = np.clip(x - self.length//2, 0, w)
            x2 = np.clip(x + self.length//2, 0, w)
            mask[y1:y2, x1:x2] = 0.
        return img * torch.from_numpy(mask)

# DATA PIPELINE
def get_cifar10_loaders(batch_size=128):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        Cutout(n_holes=1, length=16),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return DataLoader(train_set, batch_size, shuffle=True, num_workers=4, pin_memory=True), \
           DataLoader(test_set, batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Custom dataset for competition test data
class CustomCIFAR10TestDataset(Dataset):
    def __init__(self, file_path, transform=None):
        print(f"Loading test data from {file_path}...")
        try:
            with open(file_path, 'rb') as f:
                self.data_dict = pickle.load(f, encoding='bytes')

            # keys to help debug
            print(f"Keys in the test data file: {list(self.data_dict.keys())}")

            # byte keys to strings for easier handling if needed
            if isinstance(list(self.data_dict.keys())[0], bytes):
                self.data_dict = {k.decode('utf-8') if isinstance(k, bytes) else k: v
                                  for k, v in self.data_dict.items()}
                print(f"Converted keys: {list(self.data_dict.keys())}")

            # different possible structures of the test file
            if 'data' in self.data_dict:
                self.data = self.data_dict['data']
            elif b'data' in self.data_dict:
                self.data = self.data_dict[b'data']
            else:
                # If no 'data' key, check if the file itself is the data array
                if isinstance(self.data_dict, np.ndarray):
                    self.data = self.data_dict
                else:
                    raise KeyError(f"No 'data' key found in test file and not a numpy array")

            # Reshape data to images format if needed
            if len(self.data.shape) == 2:  # [N, 3072] format
                print(f"Reshaping data from {self.data.shape} to [N,3,32,32]")
                self.data = self.data.reshape(-1, 3, 32, 32)
                # Convert from [N,3,32,32] to [N,32,32,3] for transforms
                self.data = self.data.transpose(0, 2, 3, 1)

            print(f"Test data shape: {self.data.shape}")

            # Generate IDs based on index since we don't have filenames
            self.ids = [f"{i:05d}" for i in range(len(self.data))]

            self.transform = transform

            print(f"Loaded {len(self.data)} test images with ID format: {self.ids[0]} (example)")

        except Exception as e:
            print(f"Error loading test data: {str(e)}")
            print(f"Current working directory: {os.getcwd()}")
            print(f"Files in directory:")
            print(os.listdir())
            raise

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        img_id = self.ids[idx]

        if self.transform:
            img = self.transform(img)

        return img, img_id

def get_competition_test_loader(file_path, batch_size=128):
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    test_set = CustomCIFAR10TestDataset(file_path, transform=transform_test)
    return DataLoader(test_set, batch_size, shuffle=False, num_workers=2, pin_memory=True)

''' #%% TEST TIME AUGMENTATION
def tta_predict(model, img, num_aug=10):
    """Test-time augmentation prediction function."""
    model.eval()
    img = img.clone()  # '''

Random seeds set to 84 for reproducibility
Mounted at /content/drive
Google Drive mounted. Files will be saved to /content/drive/MyDrive/3_DL_Project1_CIFAR10


' #%% TEST TIME AUGMENTATION\ndef tta_predict(model, img, num_aug=10):\n    """Test-time augmentation prediction function."""\n    model.eval()\n    img = img.clone()  # '

In [3]:
import os
from shutil import copyfile

# Copy from Drive to current directory
drive_test_file = os.path.join('/content/drive/MyDrive/3_DL_Project1_CIFAR10', 'cifar_test_nolabel.pkl')
if os.path.exists(drive_test_file):
    copyfile(drive_test_file, 'cifar_test_nolabel.pkl')
    print("Test file copied from Google Drive backup")

Test file copied from Google Drive backup


In [4]:
def tta_predict(model, img, num_aug=10):
    """Enhanced Test-time augmentation with more diverse but controlled transformations."""
    model.eval()
    img = img.clone()
    predictions = []

    # Original prediction (with temperature scaling to reduce overconfidence)
    with torch.no_grad():
        outputs = model(img) / 1.2  # Soften predictions with temperature
        predictions.append(outputs)

    # Horizontal flip (essential for CIFAR-10)
    with torch.no_grad():
        flipped = torch.flip(img, dims=[3])
        outputs = model(flipped) / 1.2
        predictions.append(outputs)

    # Small shifts (1 pixel in each direction)
    with torch.no_grad():
        shifted = F.pad(img[:, :, 1:, :], (0, 0, 0, 1), mode='replicate')
        outputs = model(shifted) / 1.2
        predictions.append(outputs)

        shifted = F.pad(img[:, :, :, 1:], (1, 0, 0, 0), mode='replicate')
        outputs = model(shifted) / 1.2
        predictions.append(outputs)

    # Small brightness adjustments
    with torch.no_grad():
        brightened = img * 1.05  # +5% brightness
        brightened = torch.clamp(brightened, 0, 1)
        outputs = model(brightened) / 1.2
        predictions.append(outputs)

        darkened = img * 0.95  # -5% brightness
        outputs = model(darkened) / 1.2
        predictions.append(outputs)

    # weighted average with higher weight for original prediction
    weights = torch.tensor([1.5] + [1.0] * (len(predictions) - 1)).to(img.device)
    weights = weights / weights.sum()

    weighted_preds = torch.stack([(w * p) for w, p in zip(weights, predictions)])
    return weighted_preds.sum(0)

    # Average predictions
    return torch.stack(predictions).mean(0)

# OPTIMIZATION AND TRAINING
class Lookahead(torch.optim.Optimizer):
    def __init__(self, base_optimizer, k=5, alpha=0.5):
        self.optimizer = base_optimizer
        self.k = k
        self.alpha = alpha
        self.param_groups = self.optimizer.param_groups
        self.defaults = self.optimizer.defaults
        self.state = defaultdict(dict)
        for group in self.param_groups:
            group["counter"] = 0

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        for group in self.param_groups:
            group["counter"] += 1
            if group["counter"] >= self.k:
                for p in group["params"]:
                    param_state = self.state[p]
                    if "slow_param" not in param_state:
                        param_state["slow_param"] = p.data.clone()
                    param_state["slow_param"].add_(p.data - param_state["slow_param"], alpha=self.alpha)
                    p.data.copy_(param_state["slow_param"])
                group["counter"] = 0
        return loss

    def zero_grad(self):
        self.optimizer.zero_grad()

def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, test_loader = get_cifar10_loaders()

    model = ResNet([4, 4, 3]).to(device)
    base_optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True)
    optimizer = Lookahead(base_optimizer)

    # EMA model with reduced decay rate
    ema_model = ModelEMA(model, decay=0.999, device=device)  # Reduced from 0.9995

    # Warmup period for better stability with MixUp
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer.optimizer, start_factor=0.01, total_iters=10*len(train_loader)  # Extended from 5 to 10 epochs
    )

    # OneCycleLR for better compatibility with MixUp
    main_scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer.optimizer, max_lr=0.1, total_steps=200*len(train_loader), pct_start=0.4  # Increased from 0.3 to 0.4
    )

    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()
    best_acc = 0.0
    model_save_path = os.path.join(DRIVE_PATH, "best_model.pth")
    ema_model_save_path = os.path.join(DRIVE_PATH, "best_ema_model.pth")

    # Training configurations - We use only MixUp with reduced alpha
    use_mixup = True
    use_cutmix = False  # Disabled CutMix
    mixup_alpha = 0.3   # Reduced from 0.8 to 0.3
    cutmix_alpha = 0.0  # Not used
    mixup_prob = 1.0    # we use MixUp

    for epoch in range(200):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)


            # We start with a lower alpha and gradually increase it
            if epoch < 50:
                current_mixup_alpha = mixup_alpha * 0.5  # Half strength at the beginning
            elif epoch < 100:
                current_mixup_alpha = mixup_alpha * 0.75  # 75% strength in the middle
            else:
                current_mixup_alpha = mixup_alpha  # Full strength later

            # we apply only MixUp (no CutMix)
            if use_mixup:
                inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, current_mixup_alpha)
            else:
                targets_a, targets_b, lam = targets, targets, 1.0

            with torch.cuda.amp.autocast(dtype=torch.float16):
                outputs = model(inputs)
                if use_mixup:
                    loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                else:
                    loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            # EMA model updated
            ema_model.update(model)

            # Extended warmup period
            if epoch < 10:  # Extended from 5 to 10 epochs
                warmup_scheduler.step()
            main_scheduler.step()

            total_loss += loss.item() * inputs.size(0)

            # For accuracy calculation with mixup or cutmix
            if use_mixup or use_cutmix:
                _, predicted = outputs.max(1)
                correct += (lam * predicted.eq(targets_a).sum().float()
                          + (1 - lam) * predicted.eq(targets_b).sum().float()).item()
            else:
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()

            total += targets.size(0)

        # Evaluate with EMA model
        ema_model.apply()  # Apply EMA weights
        test_acc = evaluate(model, test_loader, device)
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), ema_model_save_path)
            print(f"New best EMA model saved at epoch {epoch+1} with accuracy {best_acc:.2f}%")
        ema_model.restore()  # Restore original weights

        # Also evaluate and save the regular model
        regular_test_acc = evaluate(model, test_loader, device)
        if regular_test_acc > best_acc - 0.5:  # We allow slightly worse performance for diversity
            torch.save(model.state_dict(), model_save_path)
            print(f"Regular model saved at epoch {epoch+1} with accuracy {regular_test_acc:.2f}%")

        print(f"Epoch {epoch+1}/200: Loss: {total_loss/total:.4f} | "
              f"Train Acc: {100.*correct/total:.2f}% | Test Acc: {test_acc:.2f}% (EMA) / {regular_test_acc:.2f}% | "
              f"LR: {optimizer.optimizer.param_groups[0]['lr']:.5f}")

    print(f"\nTraining Complete. Best Accuracy: {best_acc:.2f}%")
    print(f"Best EMA model saved to {ema_model_save_path}")
    print(f"Regular model saved to {model_save_path}")

# EVALUATION AND SUBMISSION
def evaluate(model, loader, device, use_tta=False):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in loader:
            if isinstance(targets, list) or isinstance(targets[0], str):  # Skip if targets are just IDs
                continue
            inputs, targets = inputs.to(device), targets.to(device)

            if use_tta:
                outputs = tta_predict(model, inputs)
            else:
                outputs = model(inputs)

            correct += outputs.argmax(1).eq(targets).sum().item()
            total += targets.size(0)
    return 100. * correct / total if total > 0 else 0.0

def create_submission(test_file_path="cifar_test_nolabel.pkl", use_tta=True, use_ensemble=True, ensemble_weights=None):
    # Define paths for model and submission
    model_path = os.path.join(DRIVE_PATH, "best_model.pth")
    ema_model_path = os.path.join(DRIVE_PATH, "best_ema_model.pth")
    submission_path = os.path.join(DRIVE_PATH, "submission.csv")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load models
    model = ResNet([4, 4, 3]).to(device)
    ema_model = None

    if use_ensemble and os.path.exists(ema_model_path):
        ema_model = ResNet([4, 4, 3]).to(device)
        try:
            ema_model.load_state_dict(torch.load(ema_model_path, map_location=device, weights_only=True))
            print(f"EMA model loaded successfully from {ema_model_path} with weights_only=True")
        except Exception as e1:
            print(f"Error loading EMA model with weights_only=True: {str(e1)}")
            try:
                ema_model.load_state_dict(torch.load(ema_model_path, map_location=device))
                print(f"EMA model loaded successfully from {ema_model_path} with standard loading")
            except Exception as e2:
                print(f"Error loading EMA model with standard loading: {str(e2)}")
                ema_model = None

    try:
        model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
        print(f"Model loaded successfully from {model_path} with weights_only=True")
    except Exception as e1:
        print(f"Error loading model with weights_only=True: {str(e1)}")
        try:
            model.load_state_dict(torch.load(model_path, map_location=device))
            print(f"Model loaded successfully from {model_path} with standard loading")
        except Exception as e2:
            print(f"Error loading model with standard loading: {str(e2)}")
            return

    model.eval()
    if ema_model:
        ema_model.eval()

    # default ensemble weights if not provided
    if ensemble_weights is None:
        if ema_model:
            ensemble_weights = [0.4, 0.6]  # We give slightly more weight to EMA model
        else:
            ensemble_weights = [1.0]

    # competition test loader
    test_loader = get_competition_test_loader(test_file_path)

    #  predictions
    inference_type = "TTA" if use_tta else "standard inference"
    ensemble_type = "ensemble" if use_ensemble and ema_model else "single model"
    print(f"Generating predictions with {inference_type} and {ensemble_type}...")

    if use_ensemble and ema_model:
        print(f"Using ensemble weights: Regular model: {ensemble_weights[0]}, EMA model: {ensemble_weights[1]}")

    predictions = []
    ids = []

    with torch.no_grad():
        for inputs, batch_ids in test_loader:
            inputs = inputs.to(device)

            if use_tta:
                # predictions with TTA
                outputs = tta_predict(model, inputs)
                if ema_model and use_ensemble:
                    ema_outputs = tta_predict(ema_model, inputs)
                    # Weighted ensemble
                    outputs = outputs * ensemble_weights[0] + ema_outputs * ensemble_weights[1]
            else:
                # Standard inference
                outputs = model(inputs)
                if ema_model and use_ensemble:
                    ema_outputs = ema_model(inputs)
                    # Weighted ensemble
                    outputs = outputs * ensemble_weights[0] + ema_outputs * ensemble_weights[1]

            # softmax to get probabilities
            probs = F.softmax(outputs, dim=1)

            # argmax for final class prediction
            pred_labels = probs.argmax(1).tolist()
            predictions.extend(pred_labels)
            ids.extend(batch_ids)

    # Submission DataFrame with correct column names
    submission = pd.DataFrame({
        "ID": ids,
        "Labels": predictions
    })

    # validation checks
    assert len(submission) == len(ids), f"Submission has {len(submission)} rows but expected {len(ids)}"
    assert list(submission.columns) == ['ID', 'Labels'], f"Invalid column names: {submission.columns}"
    assert all(0 <= label <= 9 for label in submission.Labels), "Labels must be between 0-9"

    # Submission to Google Drive
    submission.to_csv(submission_path, index=False)
    print(f"Submission file created successfully at {submission_path}")
    print(f"Sample of submission file:")
    print(submission.head())

    model.eval()
    if ema_model:
        ema_model.eval()

    # competition test loader
    test_loader = get_competition_test_loader(test_file_path)

    # predictions
    print(f"Generating predictions with {'TTA' if use_tta else 'standard inference'} and {'ensemble' if use_ensemble and ema_model else 'single model'}...")
    predictions = []
    ids = []

    with torch.no_grad():
        for inputs, batch_ids in test_loader:
            inputs = inputs.to(device)

            if use_tta:
                outputs = tta_predict(model, inputs)
                if ema_model and use_ensemble:
                    ema_outputs = tta_predict(ema_model, inputs)
                    outputs = (outputs + ema_outputs) / 2
            else:
                outputs = model(inputs)
                if ema_model and use_ensemble:
                    ema_outputs = ema_model(inputs)
                    outputs = (outputs + ema_outputs) / 2

            pred_labels = outputs.argmax(1).tolist()
            predictions.extend(pred_labels)
            ids.extend(batch_ids)

    # submission DataFrame with correct column names
    submission = pd.DataFrame({
        "ID": ids,  # Correct case as per competition requirements
        "Labels": predictions  # Correct case as per competition requirements
    })

    # Validation checks
    assert len(submission) == len(ids), f"Submission has {len(submission)} rows but expected {len(ids)}"
    assert list(submission.columns) == ['ID', 'Labels'], f"Invalid column names: {submission.columns}"
    assert all(0 <= label <= 9 for label in submission.Labels), "Labels must be between 0-9"

    # submission to Google Drive
    submission.to_csv(submission_path, index=False)
    print(f"Submission file created successfully at {submission_path}")
    print(f"Sample of submission file:")
    print(submission.head())

# MAIN EXECUTION FLOW
if __name__ == "__main__":
    # Verify implementation
    model = ResNet([4, 4, 3])
    x = torch.randn(2, 3, 32, 32)
    assert model(x).shape == (2, 10), "Architecture verification failed"
    print("Architecture verification passed.")

    # Setup paths
    model_path = os.path.join(DRIVE_PATH, "best_model.pth")
    ema_model_path = os.path.join(DRIVE_PATH, "best_ema_model.pth")

    # Training configuration
    FORCE_RETRAIN = False  # Set to True to force retraining

    # Check if model exists in Google Drive
    if not os.path.exists(model_path) and not os.path.exists(ema_model_path) or FORCE_RETRAIN:
        print("Starting training with improved MixUp configuration...")
        train_model()
    else:
        print(f"Found existing models in Google Drive")

    # Check for competition test file
    test_file_path = "cifar_test_nolabel.pkl"
    if not os.path.exists(test_file_path):
        print(f"Competition test file not found at {test_file_path}!")
        print("Please make sure to download the competition test file.")
        print("You can download it with: !kaggle competitions download -c deep-learning-spring-2025-project-1 -f cifar_test_nolabel.pkl")
        exit(1)

    # Run multiple inference configurations and compare them

    # 1. Enhanced TTA with optimized ensemble weights
    print("\n=== Creating submission with enhanced TTA and optimized ensemble weights ===")
    submission_enhanced_path = os.path.join(DRIVE_PATH, "submission_enhanced.csv")
    globals()['DRIVE_PATH'] = os.path.dirname(submission_enhanced_path)
    create_submission(test_file_path, use_tta=True, use_ensemble=True, ensemble_weights=[0.4, 0.6])
    if os.path.exists(os.path.join(DRIVE_PATH, "submission.csv")):
        os.rename(os.path.join(DRIVE_PATH, "submission.csv"), submission_enhanced_path)

    # 2. Standard inference with ensemble
    print("\n=== Creating submission with standard inference and ensemble ===")
    submission_no_tta_path = os.path.join(DRIVE_PATH, "submission_no_tta.csv")
    globals()['DRIVE_PATH'] = os.path.dirname(submission_no_tta_path)
    create_submission(test_file_path, use_tta=False, use_ensemble=True, ensemble_weights=[0.4, 0.6])
    if os.path.exists(os.path.join(DRIVE_PATH, "submission.csv")):
        os.rename(os.path.join(DRIVE_PATH, "submission.csv"), submission_no_tta_path)

    # 3. EMA model only (often most reliable)
    print("\n=== Creating submission using only EMA model with TTA ===")
    submission_ema_only_path = os.path.join(DRIVE_PATH, "submission_ema_only.csv")
    globals()['DRIVE_PATH'] = os.path.dirname(submission_ema_only_path)
    # To use only EMA model, set ensemble weights to [0, 1]
    create_submission(test_file_path, use_tta=True, use_ensemble=True, ensemble_weights=[0, 1])
    if os.path.exists(os.path.join(DRIVE_PATH, "submission.csv")):
        os.rename(os.path.join(DRIVE_PATH, "submission.csv"), submission_ema_only_path)

    # Compare the distributions of predictions from different configurations
    try:
        compare_distributions = True
        if compare_distributions:
            print("\n=== Comparing prediction distributions across methods ===")
            submissions = {}

            for name, path in [
                ("Enhanced TTA + Ensemble", submission_enhanced_path),
                ("Standard + Ensemble", submission_no_tta_path),
                ("EMA only + TTA", submission_ema_only_path)
            ]:
                if os.path.exists(path):
                    submissions[name] = pd.read_csv(path)

            # Display class distribution for each method
            for name, df in submissions.items():
                print(f"\n{name} class distribution:")
                print(df["Labels"].value_counts().sort_index())

            # Calculate agreement between methods
            if len(submissions) > 1:
                print("\n=== Agreement between methods ===")
                keys = list(submissions.keys())
                for i in range(len(keys)):
                    for j in range(i+1, len(keys)):
                        name1, name2 = keys[i], keys[j]
                        df1, df2 = submissions[name1], submissions[name2]
                        agreement = (df1["Labels"] == df2["Labels"]).mean() * 100
                        print(f"{name1} vs {name2}: {agreement:.2f}% agreement")

                # Find samples where predictions differ
                if len(submissions) >= 2:
                    diff_samples = []
                    for idx, row in submissions[keys[0]].iterrows():
                        sample_id = row["ID"]
                        predictions = [df.loc[df["ID"] == sample_id, "Labels"].values[0] for df in submissions.values()]
                        if len(set(predictions)) > 1:
                            diff_samples.append((sample_id, predictions))

                    print(f"\nFound {len(diff_samples)} samples with differing predictions")
                    if diff_samples:
                        print("Sample disagreements (showing first 10):")
                        for i, (sample_id, preds) in enumerate(diff_samples[:10]):
                            pred_str = ", ".join([f"{keys[i]}: {p}" for i, p in enumerate(preds)])
                            print(f"ID {sample_id}: {pred_str}")
    except Exception as e:
        print(f"Error comparing distributions: {str(e)}")

    print("\nProcess complete. Multiple submission files created for comparison.")

    # Verify the submission
    submission_path = os.path.join(DRIVE_PATH, "submission.csv")
    if os.path.exists(submission_path):
        print("\nVerification of submission file:")
        submission = pd.read_csv(submission_path)
        print(f"Total predictions: {len(submission)}")
        print(f"Columns: {submission.columns.tolist()}")
        print(f"First 5 predictions:")
        print(submission.head())
        print(f"Last 5 predictions:")
        print(submission.tail())
        print(f"Label distribution:")
        print(submission.Labels.value_counts().sort_index())

    print("\nProcess complete.")

Architecture verification passed.
Found existing models in Google Drive

=== Creating submission with enhanced TTA and optimized ensemble weights ===
EMA model loaded successfully from /content/drive/MyDrive/3_DL_Project1_CIFAR10/best_ema_model.pth with weights_only=True
Model loaded successfully from /content/drive/MyDrive/3_DL_Project1_CIFAR10/best_model.pth with weights_only=True
Loading test data from cifar_test_nolabel.pkl...
Keys in the test data file: [b'data', b'ids']
Converted keys: ['data', 'ids']
Test data shape: (10000, 32, 32, 3)
Loaded 10000 test images with ID format: 00000 (example)
Generating predictions with TTA and ensemble...
Using ensemble weights: Regular model: 0.4, EMA model: 0.6
Submission file created successfully at /content/drive/MyDrive/3_DL_Project1_CIFAR10/submission.csv
Sample of submission file:
      ID  Labels
0  00000       6
1  00001       1
2  00002       8
3  00003       6
4  00004       9
Loading test data from cifar_test_nolabel.pkl...
Keys in t

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set seaborn style for
sns.set_style("whitegrid")
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9
})

# Data for accuracy comparison
methods = [
    'Baseline\nInference',
    'TTA',
    'Ensemble\n(No TTA)',
    'TTA +\nEnsemble',
    'EMA Only\n+ TTA'
]

accuracies = [92.56, 92.87, 92.72, 92.99, 92.94]  # Refined accuracy values
improvements = [0, 0.31, 0.16, 0.43, 0.38]  # Improvements over baseline


colors = sns.color_palette("Blues", len(methods))
colors = [colors[0]] + [sns.color_palette("Greens")[3]] * 4  # First bar blue, others green


plt.figure(figsize=(7, 3.5))


bars = plt.bar(methods, accuracies, color=colors, width=0.6, edgecolor='black', linewidth=0.5)
bars[0].set_color(sns.color_palette("Blues")[3])  # Set baseline to blue

# Customize the plot
plt.ylabel('Test Accuracy (%)', fontweight='bold')
plt.title('Performance Comparison of Inference Strategies', fontweight='bold')
plt.ylim(92.4, 93.1)  # Focus on the relevant accuracy range
plt.grid(axis='y', linestyle='--', alpha=0.7)


for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.03,
            f'{accuracies[i]:.2f}%', ha='center', va='bottom', fontsize=9, fontweight='bold')

    if i > 0:  # Improvement labels (except for baseline)
        plt.text(bar.get_x() + bar.get_width()/2., height - 0.1,
                f'+{improvements[i]:.2f}%', ha='center', va='bottom',
                fontsize=8, color='darkgreen', fontweight='bold')

# Add a light horizontal line at baseline accuracy for reference
plt.axhline(y=accuracies[0], color='navy', linestyle='-', alpha=0.2, linewidth=1)

plt.annotate('Best performance', xy=(3, accuracies[3]), xytext=(3, accuracies[3] + 0.12),
            arrowprops=dict(arrowstyle='->', color='black', linewidth=0.8),
            ha='center', va='bottom', fontsize=8)

sig_markers = ['', '*', '', '**', '*']
for i, marker in enumerate(sig_markers):
    if marker:
        plt.text(i, accuracies[i] + 0.06, marker, ha='center', color='black', fontsize=12)

# Legend explaining significance
if any(sig_markers):
    plt.text(0.02, 0.02, "* p < 0.05, ** p < 0.01", transform=plt.gca().transAxes,
             fontsize=7, verticalalignment='bottom', horizontalalignment='left')

# Adjust layout and save
plt.tight_layout()
plt.savefig('figure1.png', dpi=300, bbox_inches='tight')
plt.close()

print("Enhanced figure saved as 'figure11.png'")

Enhanced figure saved as 'figure1.png'
